In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf




Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [9

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("fatema").getOrCreate()

url = 'https://raw.githubusercontent.com/FatemaGirnary/Project_4/main/Resources/heart.csv'
from pyspark import SparkFiles
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("heart.csv"), header=True)
df.show()

/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(


+---+---+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+
|Age|Sex|ChestPainType|RestingBP|Cholesterol|FastingBS|RestingECG|MaxHR|ExerciseAngina|Oldpeak|ST_Slope|HeartDisease|
+---+---+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+
| 40|  M|          ATA|      140|        289|        0|    Normal|  172|             N|      0|      Up|           0|
| 49|  F|          NAP|      160|        180|        0|    Normal|  156|             N|      1|    Flat|           1|
| 37|  M|          ATA|      130|        283|        0|        ST|   98|             N|      0|      Up|           0|
| 48|  F|          ASY|      138|        214|        0|    Normal|  108|             Y|    1.5|    Flat|           1|
| 54|  M|          NAP|      150|        195|        0|    Normal|  122|             N|      0|      Up|           0|
| 39|  M|          NAP|      120|        339|        0| 

In [3]:
# Get number of columns
num_cols = len(df.columns)

# Get number of rows
num_rows = df.count()

print(f"Number of columns: {num_cols}\nNumber of rows: {num_rows}")

Number of columns: 12
Number of rows: 918


In [4]:
from pyspark.sql.functions import count, col

# Group by all columns and count occurrences
duplicate_counts = df.groupBy(df.columns).count()

# Filter for rows with count > 1 to find duplicates
duplicates = duplicate_counts.filter(col("count") > 1)

# Count the number of duplicate rows
duplicate_count = duplicates.count()

print(f"Number of duplicate rows: {duplicate_count}")

Number of duplicate rows: 0


In [5]:

# Drop rows with any null values
df = df.dropna()

# Count the number of rows after dropping null values
row_count_after_drop = df.count()

print(f"Number of rows after dropping null values: {row_count_after_drop}")


Number of rows after dropping null values: 918


In [ ]:
df.show()

+---+---+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+
|Age|Sex|ChestPainType|RestingBP|Cholesterol|FastingBS|RestingECG|MaxHR|ExerciseAngina|Oldpeak|ST_Slope|HeartDisease|
+---+---+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+
| 40|  M|          ATA|      140|        289|        0|    Normal|  172|             N|      0|      Up|           0|
| 49|  F|          NAP|      160|        180|        0|    Normal|  156|             N|      1|    Flat|           1|
| 37|  M|          ATA|      130|        283|        0|        ST|   98|             N|      0|      Up|           0|
| 48|  F|          ASY|      138|        214|        0|    Normal|  108|             Y|    1.5|    Flat|           1|
| 54|  M|          NAP|      150|        195|        0|    Normal|  122|             N|      0|      Up|           0|
| 39|  M|          NAP|      120|        339|        0| 

In [6]:
p_df = df.toPandas()
p_df.head(10)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0,Up,0
5,39,M,NAP,120,339,0,Normal,170,N,0,Up,0
6,45,F,ATA,130,237,0,Normal,170,N,0,Up,0
7,54,M,ATA,110,208,0,Normal,142,N,0,Up,0
8,37,M,ASY,140,207,0,Normal,130,Y,1.5,Flat,1
9,48,F,ATA,120,284,0,Normal,120,N,0,Up,0


# New dataframe dropping 3 least important features from the logistics regression





In [7]:
p_df_clean = p_df.drop(columns=["ST_Slope", "RestingECG", "MaxHR"])
p_df_clean.head(10)


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,ExerciseAngina,Oldpeak,HeartDisease
0,40,M,ATA,140,289,0,N,0,0
1,49,F,NAP,160,180,0,N,1,1
2,37,M,ATA,130,283,0,N,0,0
3,48,F,ASY,138,214,0,Y,1.5,1
4,54,M,NAP,150,195,0,N,0,0
5,39,M,NAP,120,339,0,N,0,0
6,45,F,ATA,130,237,0,N,0,0
7,54,M,ATA,110,208,0,N,0,0
8,37,M,ASY,140,207,0,Y,1.5,1
9,48,F,ATA,120,284,0,N,0,0



# Logistics regression with the reduced dataframe

In [8]:
# Convert numeric columns to the correct data type
numeric_columns = ['Age', 'RestingBP', 'Cholesterol', 'FastingBS', 'Oldpeak']
p_df_clean[numeric_columns] = p_df_clean[numeric_columns].apply(pd.to_numeric, errors='coerce')

# Now, apply get_dummies to encode categorical variables
p_df_clean_encoded = pd.get_dummies(p_df_clean, columns=['Sex', 'ChestPainType', 'ExerciseAngina'], drop_first=True)

# Ensure target variable is integer
y = p_df_clean_encoded['HeartDisease'].astype(int)

# Splitting the data
X = p_df_clean_encoded.drop('HeartDisease', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
target = p_df_clean_encoded['HeartDisease']
features = p_df_clean_encoded.drop(columns=['HeartDisease']) # Use square brackets to select columns and .drop() to exclude 'HeartDisease'
X = features.values  # Features array
y = target.values    # Target array
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=5)

number_input_features = X_train.shape[1]
print(f"Number of input features: {number_input_features}")

Number of input features: 10


In [10]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(solver='liblinear', random_state=1, max_iter=200)
classifier

# Fit the model using training data
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=200, random_state=1, solver='liblinear')

In [12]:
# Make a prediction using the testing data
predictions = classifier.predict(X_test)
pd.DataFrame({"Prediction": predictions, "Actual": y_test})

,Prediction,Actual
0,1,1
1,0,0
2,1,1
3,1,1
4,0,0
...,...,...
225,1,1
226,1,1
227,1,0
228,1,1


In [13]:
# Generate a confusion matrix for the model
from sklearn.metrics import confusion_matrix

print("Confusion Matrix")
confusion_matrix(y_test, predictions)

Confusion Matrix


array([[ 80,  17],
       [ 18, 115]])

In [14]:
# Print the classification report for the model
from sklearn.metrics import classification_report

print("Classification Report:")
print(classification_report(y_test, predictions))

Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.82      0.82        97
           1       0.87      0.86      0.87       133

    accuracy                           0.85       230
   macro avg       0.84      0.84      0.84       230
weighted avg       0.85      0.85      0.85       230



Compared to the Logistics regression that used the full dataset the precision of 82% to predict a healthy heart has dropped from the original model which had a 88% accuracy. The new model also has a lower accuracy of 87% for predicting a diseased heart compared to the original model which had a 90% accuracy. Dropping the 3 lower features from the dataframe has weaken this model

# Dataframe dropping the 3 least important features from the XG boost model

In [15]:
#check original df
p_df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0,Up,0


In [16]:
df_xgb = pd.get_dummies(p_df, columns=['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope'])

# Scale numerical columns
scaler = StandardScaler()
numerical_columns = ['Age', 'RestingBP', 'Cholesterol', 'MaxHR', 'Oldpeak']

df_xgb[numerical_columns] = scaler.fit_transform(df_xgb[numerical_columns])

# Now df_encoded contains your cleaned and processed data
df_xgb

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease,Sex_F,Sex_M,ChestPainType_ASY,...,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ExerciseAngina_N,ExerciseAngina_Y,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
0,-1.433140,0.410909,0.825070,0,1.382928,-0.832432,0,False,True,False,...,False,False,False,True,False,True,False,False,False,True
1,-0.478484,1.491752,-0.171961,0,0.754157,0.105664,1,True,False,False,...,True,False,False,True,False,True,False,False,True,False
2,-1.751359,-0.129513,0.770188,0,-1.525138,-0.832432,0,False,True,False,...,False,False,False,False,True,True,False,False,False,True
3,-0.584556,0.302825,0.139040,0,-1.132156,0.574711,1,True,False,True,...,False,False,False,True,False,False,True,False,True,False
4,0.051881,0.951331,-0.034755,0,-0.581981,-0.832432,0,False,True,False,...,True,False,False,True,False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,-0.902775,-1.210356,0.596393,0,-0.188999,0.293283,1,False,True,False,...,False,True,False,True,False,True,False,False,True,False
914,1.536902,0.627078,-0.053049,1,0.164684,2.357094,1,False,True,True,...,False,False,False,True,False,True,False,False,True,False
915,0.370100,-0.129513,-0.620168,0,-0.857069,0.293283,1,False,True,True,...,False,False,False,True,False,False,True,False,True,False
916,0.370100,-0.129513,0.340275,0,1.461525,-0.832432,1,True,False,False,...,False,False,True,False,False,True,False,False,True,False


In [17]:
#Dropping the least important features for xg boost model

columns_to_drop = ['ST_Slope_Down', 'RestingECG_Normal', 'ChestPainType_TA']  # Update with correct names
df_xgb = df_xgb.drop(columns=columns_to_drop, errors='ignore') # Use errors='ignore' to skip non-existent columns
df_xgb

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease,Sex_F,Sex_M,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,RestingECG_LVH,RestingECG_ST,ExerciseAngina_N,ExerciseAngina_Y,ST_Slope_Flat,ST_Slope_Up
0,-1.433140,0.410909,0.825070,0,1.382928,-0.832432,0,False,True,False,True,False,False,False,True,False,False,True
1,-0.478484,1.491752,-0.171961,0,0.754157,0.105664,1,True,False,False,False,True,False,False,True,False,True,False
2,-1.751359,-0.129513,0.770188,0,-1.525138,-0.832432,0,False,True,False,True,False,False,True,True,False,False,True
3,-0.584556,0.302825,0.139040,0,-1.132156,0.574711,1,True,False,True,False,False,False,False,False,True,True,False
4,0.051881,0.951331,-0.034755,0,-0.581981,-0.832432,0,False,True,False,False,True,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,-0.902775,-1.210356,0.596393,0,-0.188999,0.293283,1,False,True,False,False,False,False,False,True,False,True,False
914,1.536902,0.627078,-0.053049,1,0.164684,2.357094,1,False,True,True,False,False,False,False,True,False,True,False
915,0.370100,-0.129513,-0.620168,0,-0.857069,0.293283,1,False,True,True,False,False,False,False,False,True,True,False
916,0.370100,-0.129513,0.340275,0,1.461525,-0.832432,1,True,False,False,True,False,True,False,True,False,True,False


# XG boost with dropped dataframe

In [18]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report

In [19]:
# Initialize the XGBoost model
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Convert y_train and y_test to integers
y_train = y_train.astype(int)
y_test = y_test.astype(int)

# Initialize the model with some parameters - fine-tune for better performance:
xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=3,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    use_label_encoder=False,
    eval_metric='logloss'
)

In [20]:
# Train the model
xgb_model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [09:37:59] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.05, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [21]:
# Make predictions
y_pred = xgb_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Detailed classification report
print(classification_report(y_test, y_pred))

Accuracy: 0.87
              precision    recall  f1-score   support

           0       0.83      0.86      0.84        97
           1       0.89      0.87      0.88       133

    accuracy                           0.87       230
   macro avg       0.86      0.86      0.86       230
weighted avg       0.87      0.87      0.87       230



The XG boost model with the dropped dataframe did not perform as well as the original model using the full data set. Predicting a healthy heart precision had dropped from 90% to 83% similarly predicting a diseased heart precision had dropped from 90% to 83% similarly predicting a diseased heart has dropped from 91% to 90%  with a overall lower accuracy of 87% compared to 91% in the original model